In [1]:
%matplotlib qt

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import mpl_interactions as pli
import glob
import pylinac as pl

In [4]:
# Find all the dicom images using glob 
# Load the images into a list
# Create a 3D array of the images

import os

# 0.4mm_L
# 0.4_Qr40_L
# 0.4_Qr76_L
# Pelvic_2_0.4_Qr40_L
# Pelvic_2_0.4_Qr76_L
# Pelvic_3_0.4_Qr76_L
# Pelvic_32_0.4_Qr40_L


base_dir = '/media/jericho/T7/nano_particles/images/0.4mm_L/H/'
image_paths =  glob.glob(os.path.join(base_dir, '31-*.dcm'))
image_paths1 = glob.glob(os.path.join(base_dir, '32-*.dcm'))

sorted_images = sorted(image_paths, key=lambda x: int(x.split('.')[5]))
sorted_images1 = sorted(image_paths1, key=lambda x: int(x.split('.')[5]))

images = []
for image in sorted_images[:]:
    im = pl.image.load(image)
    images.append(im.array)

images1 = []
for image in sorted_images1[:]:
    im = pl.image.load(image)
    images1.append(im.array)

images = np.array(images)
images2 = np.array(images1)


In [6]:
import matplotlib.pyplot as plt
import numpy as np

def select_regions(image, num_clicks=4):
    fig, ax = plt.subplots(dpi=300)
    ax.imshow(image, aspect=1, cmap='bone',vmin=-100,vmax=100)
    ax.axis('off')

    x_coords = []

    def onclick(event):
        if len(x_coords) >= num_clicks:
            return
        ix = int(event.xdata)
        x_coords.append(ix)

        plt.axvline(x=ix, color='r', linestyle='--')
        plt.text(ix, 0, 'ROI: '+str(len(x_coords)), color='red', fontsize=12, va='bottom', ha='right')
        fig.canvas.draw()

    # Add a colorbar
    cbar = plt.colorbar(ax.imshow(image, aspect=1, cmap='bone',vmin=-100,vmax=100),orientation='horizontal')
    cbar.set_label('Pixel Value HU')

    cid = fig.canvas.mpl_connect('button_press_event', onclick)

    plt.show()
    return x_coords

# Use the function
x_coords = select_regions(images2.T[256])

In [8]:
def select_regions(image, radius=20, num_regions=5):
    fig, ax = plt.subplots(figsize=[6,4])
    ax.imshow(image, aspect='auto', cmap='turbo',vmin=5,vmax=25)
    ax.axis('off')
    mask = np.zeros_like(image)

    coords = []

    def circular_mask(x, y, radius, shape):
        xx, yy = np.ogrid[:shape[0], :shape[1]]
        circle = (xx - x) ** 2 + (yy - y) ** 2
        mask = circle < radius ** 2
        return mask

    def onclick(event):
        if len(coords) >= num_regions:
            return
        ix, iy = int(event.xdata), int(event.ydata)
        coords.append((ix, iy))

        circle = plt.Circle((ix, iy), radius, color='g', fill=False)
        
        ax.add_artist(circle)
        # Add text to label the circles
        plt.text(ix, iy, str(len(coords)), color='red', fontsize=12, va='bottom', ha='right')

        mask[circular_mask(iy, ix, radius, image.shape)] = len(coords)
        # mask[iy-radius:iy+radius, ix-radius:ix+radius] = len(coords)
        fig.canvas.draw()

    cbar = plt.colorbar(ax.imshow(image, aspect=1, cmap='turbo',vmin=5,vmax=25),orientation='horizontal')
    cbar.set_label('Pixel Value HU')
    cid = fig.canvas.mpl_connect('button_press_event', onclick)
    # plt.tight_layout()
    plt.show()

    return mask

masks= []

for x_coord in x_coords:

    # Use the function
    masks.append(select_regions(images[x_coord-20:x_coord+20].mean(axis=0)))
    # pause until the figure is closed

In [10]:
# Generate 4 subplots showing the four masks overlayed on the image using nans for zero in the masks
fig, ax = plt.subplots(2,2, figsize=[8,8])
ax = ax.ravel()
titles = ['Ag low conc.', 'Ag high conc.', 'Ag-Bi low conc.', 'Ag-Bi high conc.']
for i, (mask,x_coord) in enumerate(zip(masks,x_coords)):
    image = images2[x_coord-20:x_coord+20].mean(axis=0)
    mask_temp = mask.copy()
    mask_temp[mask==0] = np.nan
    ax[i].imshow(image, aspect='auto', cmap='turbo',vmin=5,vmax=25)
    ax[i].imshow(mask_temp, aspect='auto', cmap='tab20',vmin=0,vmax=5)
    ax[i].axis('off')
    ax[i].set_title(titles[i])

In [42]:
# Calculate the mean pixel value for each mask
mean_values = []
std_values = []

for x_coord, mask in zip(x_coords, masks):
    mean_values.append([])
    for integer in np.unique(mask):
        if integer == 0:
            continue
        temp_mean = []
        for index in range(x_coord-20, x_coord+20):
            temp_mean.append(images2[index][mask == integer].mean())
        mean_values[-1].append(np.mean(temp_mean))
        std_values.append(np.std(temp_mean))

# Plot the mean values with error bars
fig, ax = plt.subplots(dpi=300,figsize=[6,4])
colors = ['forestgreen', 'darkorange']
conc = np.array([0,0.01,0.05,0.1,0.15,0.2,0.25,0.3,0.6,1])

import colorsys
import matplotlib

def adjust_color_lightness(color, amount=0.5):
    try:
        c = matplotlib.colors.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*matplotlib.colors.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], max(0, min(1, amount * c[1])), c[2])

# Use the function
titles = ['Ag', 'Ag-Bi']
for ii, (mean, std) in enumerate(zip(mean_values[::2], std_values[::2])):
    mean_both = mean_values[ii*2] + mean_values[ii*2+ 1]
    std_both = std_values[ii*2] + std_values[ii*2+ 1]
    darker_color = adjust_color_lightness(colors[ii], 0.75)  # Darker red
    lighter_color = adjust_color_lightness(colors[ii], 1.25)  # Lighter red
    ax.errorbar(conc+0.01*(ii-1), mean_both, yerr=std_both, fmt='*', color=darker_color, ecolor=lighter_color,capsize=5, capthick=2)
    # Add a trend line
    z = np.polyfit(conc+0.01*(ii-1), mean_both, 1)
    p = np.poly1d(z)
    ax.plot(conc+0.01*(ii-1), p(conc+0.01*(ii-1)), "--", color=colors[ii],label=titles[ii])

# plt.colorbar()
plt.xlabel('Concentration (mg/mL)')
plt.ylabel('Mean Pixel Value (HU)')
plt.legend()
plt.title('Mean HU vs Conc. 40 slices 70-140keV')
plt.tight_layout()

# Save the figure
plt.savefig('40_slices_70-140keV.png', dpi=300)

In [43]:
# Calculate the mean pixel value for each mask
mean_values = []
std_values = []

for x_coord, mask in zip(x_coords, masks):
    mean_values.append([])
    for integer in np.unique(mask):
        if integer == 0:
            continue
        temp_mean = []
        for index in range(x_coord-20, x_coord+20):
            temp_mean.append(images[index][mask == integer].mean())
        mean_values[-1].append(np.mean(temp_mean))
        std_values.append(np.std(temp_mean))

# Plot the mean values with error bars
fig, ax = plt.subplots(dpi=300,figsize=[6,4])
colors = [ 'indianred', 'olivedrab']
conc = np.array([0,0.01,0.05,0.1,0.15,0.2,0.25,0.3,0.6,1])

import colorsys
import matplotlib

def adjust_color_lightness(color, amount=0.5):
    try:
        c = matplotlib.colors.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*matplotlib.colors.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], max(0, min(1, amount * c[1])), c[2])

# Use the function
titles = ['Ag', 'Ag-Bi']
for ii, (mean, std) in enumerate(zip(mean_values[::2], std_values[::2])):
    mean_both = mean_values[ii*2] + mean_values[ii*2+ 1]
    std_both = std_values[ii*2] + std_values[ii*2+ 1]
    darker_color = adjust_color_lightness(colors[ii], 0.75)  # Darker red
    lighter_color = adjust_color_lightness(colors[ii], 1.25)  # Lighter red
    ax.errorbar(conc+0.01*(ii-1), mean_both, yerr=std_both, fmt='*', color=darker_color, ecolor=lighter_color,capsize=5, capthick=2)
    # Add a trend line
    z = np.polyfit(conc+0.01*(ii-1), mean_both, 1)
    p = np.poly1d(z)
    ax.plot(conc+0.01*(ii-1), p(conc+0.01*(ii-1)), "--", color=colors[ii],label=titles[ii])

# plt.colorbar()
plt.xlabel('Concentration (mg/mL)')
plt.ylabel('Mean Pixel Value (HU)')
plt.legend()
plt.title('Mean HU vs Conc. 40 slices 20-70keV')
plt.tight_layout()

# plt.savefig('40_slices_20-70keV.png')

In [44]:
import pickle

with open('plot_data_20-70.pkl', 'wb') as f:
    pickle.dump((mean_values, std_values, conc, colors, titles), f)

In [40]:
W = 40
L = 20

fig = plt.figure(figsize=(13, 7))
plt.subplot(131)
cc= pli.hyperslicer(images,autoscale_cmap=False,cmap='bone',vmin=L-W/2,vmax=L+W/2)
plt.axis('off')
plt.title(f'20-70 keV W/L {W}/{L}')
plt.colorbar(orientation='horizontal')
plt.subplot(132)
c2 = pli.hyperslicer(images2,autoscale_cmap=False,cmap='bone',controls=cc,vmin=L-W/2,vmax=L+W/2)
plt.axis('off')
plt.title(f'70-140 keV W/L {W}/{L}')
plt.colorbar(orientation='horizontal')
plt.subplot(133)
c3 = pli.hyperslicer(images - images2,autoscale_cmap=False,cmap='bwr',controls=cc,vmin=-W/4,vmax=W/4)
plt.axis('off')
plt.title('Difference [HU]')
plt.colorbar(orientation='horizontal')
plt.tight_layout()

In [41]:
cc.save_animation('low_high_70keV_difference.gif',fig,'axis0',interval=20)

In [36]:
import mpl_interactions.ipyplot as iplt

slice = x_coords[1] #1080 #253 #787 #342 #1000

nslice = 20

x = np.linspace(0, np.pi, 200)
y = np.linspace(0, 10, 200)
X, Y = np.meshgrid(x, y)

im_hi = np.array(images2[slice-nslice:slice+nslice]).mean(axis=0)
im_low = np.array(images[slice-nslice:slice+nslice]).mean(axis=0)

# im1 = im1 - im1.min()
# im1 = im1 / im1.max()

# im2 = im2 - im2.min()
# im2 = im2 / im2.max()

def f(param1):
    temp_im = (im_low- im_hi* param1 )
    # define a standard scaler function
    def standard_scaler(x):
        return (x - x.mean()) / x.std()
    return standard_scaler(temp_im[150:-150,150:-150])

fig, ax = plt.subplots(figsize=(8,8))
controls = iplt.imshow(f, param1=(0, 3),vmin_vmax=("r", -2, 2),cmap='turbo')
plt.axis('off')
plt.colorbar(orientation='horizontal',label='Standardized Pixel Value')


with controls:
    # directly using string formatting
    # the formatting is performed in the update
    iplt.title(title="20-70keV  - 70-140keV*{param1:.2f}")


/tmp/ipykernel_46257/908404925.py:27: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(8,8))


In [37]:
controls.save_animation('weighted_subtraction.mp4',fig,'param1',interval=20)
controls.save_animation('weighted_subtraction.gif',fig,'param1',interval=20)

In [76]:
from mpl_interactions import image_segmenter
segmenter = image_segmenter(im2, mask_colors="red", mask_alpha=0.76, figsize=(7, 7))
display(segmenter)
plt.show()

In [79]:
diffs = []

for param1 in np.linspace(0, 2, 100):
    im_temp = f(param1)
    mean_low = im_temp[mask_water==1]
    mean_high = im_temp[mask_high_conc_ag==1]
    diffs.append(np.abs(mean_high.mean() - mean_low.mean())/np.sqrt(mean_high.var() + mean_low.var()))

In [81]:
plt.figure()
plt.plot(np.linspace(0, 2, 100), diffs)
plt.xlabel('weighting of 70-140keV')
plt.ylabel('CNR')
plt.title('CNR 1.0 mg/ml AG (70-140keV*weight - 20-70keV)')

Text(0.5, 1.0, 'CNR 1.0 mg/ml AG (70-140keV*weight - 20-70keV)')

In [78]:
# np.save('mask_low_conc.npy', mask_low_conc)
# np.save('mask_high_conc.npy', mask_high_conc)
# np.save('mask_water.npy', mask_water)
# np.save('mask_high_conc_ag.npy', mask_high_conc_ag)

In [77]:
mask_high_conc_ag = segmenter.mask

In [75]:
total_mask = mask_water + 2*mask_low_conc + 3 *mask_high_conc
total_mask[total_mask==0] = np.NaN

plt.figure(dpi=400)
plt.imshow(im2,cmap='bone',vmin = np.mean(im2[mask_water==1]) - 2*np.std(im2[mask_water==1]), vmax = np.mean(im2[mask_high_conc==1]) + 2*np.std(im2[mask_water==1]))
plt.imshow(total_mask, cmap='flag',alpha=0.5)
plt.axis('off')
plt.title('Image Segments')

Text(0.5, 1.0, 'Image Segments')